# Import packages and modules

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Import labeled data set

In [2]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()

-------------,ID,SourceID,RAdeg,DEdeg,Per,R21,phi21,T0,gmag,rmag,...,phi21_r,R2_g,R2_r,Amp_g,Amp_r,log(FAP_g),log(FAP_r),Type,Dmin_g,Dmin_r
0,ZTFJ000000.14+721413.7,2,0.00061,72.23716,0.299150,0.263,6.308,58388.255579,19.613,18.804,...,6.308,0.624,0.873,0.540,0.438,-13.490,-27.331,EW,0.190,0.078
1,ZTFJ000000.19+320847.2,3,0.00080,32.14645,0.287059,0.010,8.024,58280.478081,15.311,14.610,...,8.024,0.940,0.977,0.219,0.197,-7.506,-10.079,EW,0.020,0.017
2,ZTFJ000000.26+311206.3,4,0.00109,31.20176,0.362217,0.132,6.281,58283.461994,16.350,15.844,...,6.281,0.951,0.960,0.233,0.226,-7.830,-9.245,EW,0.013,0.020
3,ZTFJ000000.30+711634.1,5,0.00125,71.27616,0.268515,0.160,5.236,58657.423517,19.144,17.875,...,5.236,0.363,0.623,0.173,0.154,-9.865,-22.037,EW,0.000,0.005
4,ZTFJ000000.30+233400.5,6,0.00125,23.56682,0.269874,0.193,6.302,58437.268664,17.890,16.944,...,6.302,0.910,0.976,0.373,0.352,-7.075,-8.819,EW,0.098,0.034


# Column explanations

In [3]:
header_pd.head(27)

,--------,-------,----,-------------,-------------------------------------------
0,1- 22,A22,---,ID,ZTF variables catalog identifier
1,24- 29,I6,---,SourceID,Internal source identifier
2,31- 39,F9.5,deg,RAdeg,Right Ascension in decimal degrees (J2000
3,41- 49,F9.5,deg,DEdeg,Declination in decimal degrees (J2000)
4,51- 61,F11.7,d,Per,Period
5,63- 68,F6.3,---,R21,Ratio of a_2_/a_1_
6,70- 74,F5.3,---,phi21,The {phi}_2_ - 2*{phi}_1_
7,76- 88,F13.7,d,T0,"HJD of Minimum, HJD-2,400,000"
8,90- 95,F6.3,mag,gmag,Mean ZTF g band magnitude
9,97-102,F6.3,mag,rmag,Mean ZTF r band magnitude


# Get complete light curve for each row of labeled dataset

In [16]:
lightcurves = {} # empty dictionary to hold dataframe for each light curve
circle_radius = 0.00028 # 1 arcsec = 0.00028 degress
t_format = "ipac_table"
table_format = "FORMAT=" + str(t_format)
flag_mask = 32768
mask = "BAD_CATFLAGS_MASK=" + str(flag_mask)
collect="COLLECTION="+"ztf_dr2"
numobs = "NOBS_MIN=20"
filter_band = "g"
for i in range(0,100):
# for i in range(0,len(labeled_data.index)):
    
    ra = labeled_data.RAdeg[i]
    dec = labeled_data.DEdeg[i]
    
    circle = "POS=CIRCLE"+"+"+str(ra)+"+"+str(dec)+"+"+str(circle_radius)
    band = "BANDNAME="+ filter_band
    params = circle + "&" +  mask + "&" + numobs + "&" + collect + "&" + table_format
    
    url= "https://irsa.ipac.caltech.edu/cgi-bin/ZTF/nph_light_curves?" + params
    data = pd.read_csv(url, header=None, delim_whitespace=True, skiprows=55) # extract data
    header = pd.read_csv(url, header=None, sep='|', skiprows=50,usecols=range(1,25), nrows=1)
    data.columns = header.iloc[0].str.strip()
    lightcurves [str(labeled_data.SourceID[i])] = data
#     print("row "+str(i) + (" unique filter extracted"))
#     print(data.filtercode.unique())

# Calculate Features

In [17]:
# weighted mean
def weighted_mean(mag,mag_err):
    mag2 = (mag_err*mag_err) # mag err square
    mag2_inv = 1/mag2.values; # take inverse of the values
    w = pd.Series(mag2_inv) # covert it back to s series
    sw = w.sum() # sum of weights
    wmag = mag*w # multiply magnitude with weights
    wmean = wmag.sum()/sw # weighted mean
    return wmean

In [18]:
# welsh J, K statistics
def welsh_staton(mag_series,wmean):
    N = len(mag_series)
    d_i = N/(N-1)*(mag_series - wmean) # replace mean by weighted mean
    d_i1 = d_i.shift(periods=-1)
    d_i1.fillna(0, inplace = True)
    Pi = d_i*d_i1
    Pi_val = Pi.values
    Psign = np.sign(Pi_val)
    Jval = Psign*np.sqrt(np.abs(Pi_val))
    J = np.sum(Jval) 
    K1 = abs(d_i.values)/N
    K2 = np.sqrt(1/N*np.sum(d_i.values*d_i.values))
    K = np.sum(K1*K2)
    return J, K 

In [19]:
g_mean = []
g_wmean = [] # weighted mean
g_MAD = []
g_IQR = []
g_f60 = []
g_f70 = []
g_f80 = []
g_f90 = []
g_skew = []
g_kurtosis = []
g_welsh_K = []
g_welsh_J = []

r_mean = []
r_wmean = [] # weighted mean
r_MAD = []
r_IQR = []
r_f60 = []
r_f70 = []
r_f80 = []
r_f90 = []
r_skew = []
r_kurtosis = []
r_welsh_K = []
r_welsh_J = []


for lc in lightcurves:
    df = lightcurves[lc]
    # split df by filtercode
    dfg = df.loc[df["filtercode"] == "zg"]
    dfr = df.loc[df["filtercode"] == "zr"]
    
    if dfg is not None:
        N = len(df)
        wmean_temp = weighted_mean(df.mag,df.magerr)
        K_temp, J_temp =  welsh_staton(df.mag, wmean_temp )
        g_mean.append(df.mag.mean())
        g_wmean.append(wmean_temp) 
        deviation = abs(df.mag - df.mag.median())
        g_MAD.append(deviation.median())
        g_IQR.append(df.mag.quantile(0.75) - df.mag.quantile(0.25))
        g_f60.append(df.mag.quantile(0.80) - df.mag.quantile(0.2))
        g_f70.append(df.mag.quantile(0.85) - df.mag.quantile(0.15))
        g_f80.append(df.mag.quantile(0.9) - df.mag.quantile(0.10))
        g_f90.append(df.mag.quantile(0.95) - df.mag.quantile(0.05))
        g_skew.append(df.mag.skew())
        g_kurtosis.append(df.mag.kurtosis())
        g_welsh_J.append(J_temp)
        g_welsh_K.append(K_temp)
    else:
        g_mean.append(np.NaN)
        g_wmean.append(np.NaN) 
        g_MAD.append(np.NaN)
        g_IQR.append(np.NaN)
        g_f60.append(np.NaN)
        g_f70.append(np.NaN)
        g_f80.append(np.NaN)
        g_f90.append(np.NaN)
        g_skew.append(np.NaN)
        g_kurtosis.append(np.NaN)
        g_welsh_J.append(np.NaN)
        g_welsh_K.append(np.NaN)
        
    if dfr is not None:
        N = len(df)
        wmean_temp = weighted_mean(df.mag,df.magerr)
        K_temp, J_temp =  welsh_staton(df.mag, wmean_temp )
        r_mean.append(df.mag.mean())
        r_wmean.append(wmean_temp) 
        deviation = abs(df.mag - df.mag.median())
        r_MAD.append(deviation.median())
        r_IQR.append(df.mag.quantile(0.75) - df.mag.quantile(0.25))
        r_f60.append(df.mag.quantile(0.80) - df.mag.quantile(0.2))
        r_f70.append(df.mag.quantile(0.85) - df.mag.quantile(0.15))
        r_f80.append(df.mag.quantile(0.9) - df.mag.quantile(0.10))
        r_f90.append(df.mag.quantile(0.95) - df.mag.quantile(0.05))
        r_skew.append(df.mag.skew())
        r_kurtosis.append(df.mag.kurtosis())
        r_welsh_J.append(J_temp)
        r_welsh_K.append(K_temp)
    else:
        r_mean.append(np.NaN)
        r_wmean.append(np.NaN) 
        r_MAD.append(np.NaN)
        r_IQR.append(np.NaN)
        r_f60.append(np.NaN)
        r_f70.append(np.NaN)
        r_f80.append(np.NaN)
        r_f90.append(np.NaN)
        r_skew.append(np.NaN)
        r_kurtosis.append(np.NaN)
        r_welsh_J.append(np.NaN)
        r_welsh_K.append(np.NaN)

    
features = pd.DataFrame()
# g filter data
features['g_mean'] = g_mean
features['g_wmean'] = g_wmean
features['g_MAD'] = g_MAD
features['g_IQR'] = g_IQR
features['g_f60'] = g_f60
features['g_f70'] = g_f70
features['g_f80'] = g_f80
features['g_f90'] = g_f90
features['g_skew'] = g_skew
features['g_kurtosis'] = g_kurtosis
features['g_welsh_J'] = g_welsh_J
features['g_welsh_K'] = g_welsh_K

# r filter data
features['r_mean'] = r_mean
features['r_wmean'] = r_wmean
features['r_MAD'] = r_MAD
features['r_IQR'] = r_IQR
features['r_f60'] = r_f60
features['r_f70'] = r_f70
features['r_f80'] = r_f80
features['r_f90'] = r_f90
features['r_skew'] = r_skew
features['r_kurtosis'] = r_kurtosis
features['r_welsh_J'] = r_welsh_J
features['r_welsh_K'] = r_welsh_K

features['sourceid'] = lightcurves.keys()
features.set_index('sourceid')

,g_mean,g_wmean,g_MAD,g_IQR,g_f60,g_f70,g_f80,g_f90,g_skew,g_kurtosis,...,r_MAD,r_IQR,r_f60,r_f70,r_f80,r_f90,r_skew,r_kurtosis,r_welsh_J,r_welsh_K
sourceid,,,,,,,,,,,,,,,,,,,,,
2,19.178123,18.951892,0.364013,0.811133,0.876310,0.980443,1.078816,1.232814,0.247762,-1.400375,...,0.364013,0.811133,0.876310,0.980443,1.078816,1.232814,0.247762,-1.400375,0.197777,131.561187
3,14.938193,14.820279,0.184537,0.688122,0.752726,0.808647,0.863690,0.893147,0.137726,-1.859579,...,0.184537,0.688122,0.752726,0.808647,0.863690,0.893147,0.137726,-1.859579,0.131604,43.189480
4,16.082003,15.994190,0.202108,0.516566,0.559140,0.605087,0.653707,0.695344,0.157715,-1.724736,...,0.202108,0.516566,0.559140,0.605087,0.653707,0.695344,0.157715,-1.724736,0.070001,30.519583
5,18.475873,18.062889,0.211990,1.264793,1.298252,1.335579,1.387874,1.465062,0.113854,-1.933767,...,0.211990,1.264793,1.298252,1.335579,1.387874,1.465062,0.113854,-1.933767,0.464380,242.170719
6,17.392528,17.153603,0.344473,0.941131,1.011446,1.095423,1.185724,1.264093,0.106708,-1.750476,...,0.344473,0.941131,1.011446,1.095423,1.185724,1.264093,0.106708,-1.750476,0.255293,54.508109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,17.147131,16.581076,0.121826,1.853672,1.879015,1.902491,1.929484,1.970651,0.084065,-1.989302,...,0.121826,1.853672,1.879015,1.902491,1.929484,1.970651,0.084065,-1.989302,0.983390,553.480382
98,17.150548,17.097019,0.192566,0.344553,0.365724,0.388338,0.420449,0.565247,0.316070,-0.996293,...,0.192566,0.344553,0.365724,0.388338,0.420449,0.565247,0.316070,-0.996293,0.037280,45.619397
99,12.962705,12.940647,0.246595,0.974975,1.004391,1.049454,1.118640,1.216642,0.116033,-1.910624,...,0.246595,0.974975,1.004391,1.049454,1.118640,1.216642,0.116033,-1.910624,0.258309,61.760932
